In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ.get('riot_api_key')


In [2]:
import pygsheets 
google_url = os.environ.get('google_url')
service_account = pygsheets.authorize(service_account_file = 'JSONS\\spreadsheet-automator-447604-0758cb3da318.json')
sheet = service_account.open_by_url(google_url)

In [3]:
import pandas as pd 
import requests
import time
def getLadder(rank,division):
    divisions = ['II','III','IV']
    page = pd.DataFrame()
    root = 'https://na1.api.riotgames.com/'
    meta = f'lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page=1'
    api_url = root + meta + '&api_key=' + api_key
    api_response = requests.get(api_url)
    api_df = pd.DataFrame(api_response.json())
    page = pd.concat([page,api_df])
    page_num = 2 
    while not api_df.empty:
        meta = f'lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page={page_num}'
        api_url = root + meta + '&api_key=' + api_key
        api_response = requests.get(api_url)
        api_df = pd.DataFrame(api_response.json())
        page = pd.concat([page,api_df])
        page_num+=1
        time.sleep(1)
    return page

In [4]:
def get_puuid(summoner_id):
    root = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/'
    api_url = root + summoner_id + '?api_key=' + api_key
    page = requests.get(api_url)
    return page.json()['puuid']

In [5]:
if os.path.exists('ironLadder.csv'):
    ironLadder = pd.read_csv('ironLadder.csv')
else:
    ironLadder = getLadder('IRON','I').sort_values('leaguePoints',ascending=False).reset_index(drop=True)
    ironLadder.to_csv('ironLadder.csv',index=False)
    print('Not found')

In [6]:
def getMatch(matchID):
    root = 'https://americas.api.riotgames.com/lol/match/v5/matches/'
    middle = '?api_key='
    api_url = root + matchID + middle + api_key
    time.sleep(1)
    result = requests.get(api_url)
    if result.status_code != 200:
        raise Exception("Status Code:",result.status_code)
    
    return result.json()

In [7]:
def getListMatches(puuid,start,count):
    root = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/'
    middle = '/ids?type=ranked&'
    part = 'start='+str(start)+'&count='+str(count)
    api_url = root +str(puuid)+middle + part + '&api_key=' + api_key
    page = requests.get(api_url).json()
    return page 

In [8]:
def getKDA(information,puuid):
    kda = 0
    try:
        participant = information['metadata']['participants'].index(puuid)
        kills = information['info']['participants'][participant]['kills']
        deaths = information['info']['participants'][participant]['deaths']
        assists = information['info']['participants'][participant]['assists']
        #print(information['info']['participants'][participant]['championName'])
        return (kills+assists)/max(deaths,1)
    except Exception as e:
        print(information)
        print("Exception",e)


In [9]:
def getCS(information,puuid):
    participant = information['metadata']['participants'].index(puuid)
    cs = information['info']['participants'][participant]['totalMinionsKilled']
    minutes = int(information['info']['gameDuration'])/60
    #print(cs/minutes)
    return cs/minutes

In [ ]:
def getRank(puuid):
    root_summoner = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/'
    summoner_api = root_summoner + puuid + '?api_key=' +api_key
    json = requests.get(summoner_api).json
    summoner_id = json['id']
    url = 'https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/3ai6QdnIOPfEzwy7sYkVOi56s7PL_8bQUpC7csAdOdCSBBjd?api_key=RGAPI-2f8a1ee5-ee95-4d42-a15e-d545e08b9e5e'
    root_rank = 'https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/'
    rank_api = root_rank + summoner_api + '?api_key=' + api_key
    json1 = requests.get(rank_api).json
    return json1['tier'],json1['rank']

In [20]:
def createPlayerDataBase():
    if os.path.exists('playerDataBase.csv'):
        playerDataBase = pd.read_csv('playerDataBase.csv')
    else:
        playerDataBase = pd.DataFrame({'puuid':[],'rank':[],'KDA':[],'CS/Min':[]})    
    return playerDataBase

In [18]:
def createMatchDataBase():
    if os.path.exists('matchDataBase.csv'):
        matchDataBase = pd.read_csv('matchDataBase.csv')
    else:
        matchDataBase = pd.DataFrame({'matchId':[],
                                      'player1':[],'rank':[],'KDA':[],'CS/Min':[],'Mastery':[],
                                      'player2':[],'rank':[],'KDA':[],'CS/Min':[],'Mastery':[],
                                      'player3':[],'rank':[],'KDA':[],'CS/Min':[],'Mastery':[],
                                      'player4':[],'rank':[],'KDA':[],'CS/Min':[],'Mastery':[],
                                      'player5':[],'rank':[],'KDA':[],'CS/Min':[],'Mastery':[],
                                      'player6':[],'rank':[],'KDA':[],'CS/Min':[],'Mastery':[],
                                      'player7':[],'rank':[],'KDA':[],'CS/Min':[],'Mastery':[],
                                      'player8':[],'rank':[],'KDA':[],'CS/Min':[],'Mastery':[],
                                      'player9':[],'rank':[],'KDA':[],'CS/Min':[],'Mastery':[],
                                      'player10':[],'rank':[],'KDA':[],'CS/Min':[],'Mastery':[],
                                      })
    return matchDataBase

In [11]:
#First get 1 match ID
#For each player in said match, if already in database, write down puuid and get match history from last ten matches, appending to player database their stats[Rank,KDA,CS/min]
#In each match, champion mastery, [Rank,KDA,Cs/min,win/lose] is placed in order from top to bottom in the match database
#Get a random match from a list of 10 and save the match id
#repeat until 10000 entries in match database 

In [ ]:
#Given matchID, look at all the players and see if you need to add them to the playerDataBase
#Returns a new matchID
def addToDataBase(matchID,playerDataBase):
    single_match = getMatch(matchID)
    participant_names = single_match['metadata']['participants']
    participant_datas = single_match['info']['participants']
    playerMiniBase = pd.DataFrame
    for participant_name in participant_names:
        if not participant_name in playerDataBase['puuid'].values:
            matches = getListMatches(participant_name,start = 10,count = 10)
            kda_mean = 0 
            cs_mean = 0 
            for m in matches:
                information = getMatch(m)
                kda = getKDA(information,participant_name)
                cs = getCS(information,participant_name)
                kda_mean += kda
                cs_mean += cs
            tier,tier_num = getRank(participant_name)
            kda_mean= kda_mean/len(matches)
            cs_mean = cs_mean/len(matches)
            new_row = pd.DataFrame({'puuid':[participant_name],'rank':[rank],'KDA':[kda_mean],'Cs/Min':[cs_mean]})
        playerMiniBase = pd.concat([playerMiniBase,new_row],ignore_index=True)

In [ ]:
def fillDataBase(matchID):
    matchDataBase = createMatchDataBase()
    playerDataBase = createPlayerDataBase()
    while len(matchDataBase) < 10000:
        new_line,newMatchID = addToDataBase(matchID,playerDataBase)
        
        

In [ ]:
createMatchDataBase('NA1_5206345700')

['qz_Xlpl6K90kiWzx6wBHhLIjsZKCt1RsJSI7UKtxzUB2ipvlkhxxanLt6srwC-lsIYTYchsapXAFig', 'eqrp-WeBQFTsq5w6-3QrX_27CAo8Jup6bawyrw9Y5M24lyrkkPPnQTDQ4Oyda25drlvp4MjZg5-3aQ', 'b6neNj_YKfGRuREW1LmaoTMWKF7cYdBc107QngNysELCCf0G_O7fZCnmMiXSDCw8lW6dee1EEWt4Tg', 'Tfnc-ixkF3TJ-6KcpSWfP8FEn1QfoeKnpGY3xcIQPiutac_l4Z_-HKEooa1EWx6lYDOz13vPZBSj9w', 'bNb9sdI2l0xqFDHUD_6E60qoCFjo7xNe4xiFhHKL6IsL1LlDW1VUjytR9W3EveIT1ZU-Q7RNfAMjxA', 'SfYpikaXAuZZEqyOC4A3WakIbR_lYb8VQapRhI0t6-fMPspA6T9OHdDSA8YlE17u2mvZxAbOCSqgIg', 'WftUymJJaAHwyZepZ-CnrPeHEYpDG6GehFghJ5Okq6L2uCda2_7QUViX_bfKv_P-2lc1QKD3nHmYWg', 'tvxSymXYy7dfzv0DTqnoEXc6C8s5FFZBRehTzN5KtYZSOcn4c3lJg17sZGMYvH_j5bO_GRI0ghEZEg', '9fUKh6vk2ewVEpyDznEniYFgJmHqiNnFnbXq5dOsIBWNoKEf0XEqWBjA5HAY1rSBVmne-EhEMexg_Q', 'PA-GMefU3a_H-n09lA-gA27L7Au7iLEW7KfF4yUr2z8Lb5D3f4NWPV2f7zC04jSvJd_kiJpgdExIWw']
{'allInPings': 0, 'assistMePings': 0, 'assists': 20, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 3, 'challenges': {'12AssistStreakCount': 0, 'HealFromMapSources': 0, 'Inferna

In [12]:
def appendStats():
    average_iron_1 = pd.DataFrame({'puuid':[],'KDA':[],'CS/Min':[]})
    for i in ironLadder.itertuples():
        time.sleep(10)
        puuid = get_puuid(i.summonerId)
        matches = getListMatches(puuid=puuid,start = 10,count = 10)
        kda_mean = 0 
        cs_mean = 0 
        for m in matches:
            information = getMatch(m)
            kda = getKDA(information,puuid)
            cs = getCS(information,puuid)
            kda_mean += kda
            cs_mean += cs
        kda_mean= kda_mean/len(matches)
        cs_mean = cs_mean/len(matches)
        new_row = pd.DataFrame({'puuid':[puuid],'KDA':[kda_mean],'CS/Min':[cs_mean]})
        average_iron_1 = pd.concat([average_iron_1,new_row],ignore_index=True)
        if len(average_iron_1) % 10 == 0:
            print(len(average_iron_1))
            time.sleep(120)


10
20
30
40


KeyboardInterrupt: 

In [ ]:
ironLadder['summonerId'] = ironLadder['summonerId'].map(average_iron_1)